In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/graph

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-08 19:26:40--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2020-11-08 19:26:40 (8.49 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

### Create DataFrames to match tables

In [6]:
# Read in the Review dataset as a DataFrame
from pyspark.sql.functions import col, count, to_date

In [7]:
# Create the customers_table DataFrame
customers_df = df.groupby(col("customer_id")).agg(count(col("customer_id"))).withColumnRenamed("count(customer_id)", "customer_count")

In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()

In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [10]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [11]:
total_votes = vine_df.filter(col("total_votes") > 20)
total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R2V9VX4YQ8TQ24|          5|           33|         36|   N|     

In [12]:
helpful_df = total_votes.filter((col("helpful_votes")/col("total_votes")) >= .5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R2V9VX4YQ8TQ24|          5|           33|         36|   N|                Y|
|R2UCJ0GANHZVA8|          5|           21|         21|   N|                Y|
| R8MJZ3925MBK0|          5|           96|         98|   N|     

In [13]:
vine_paid_df = vine_df.filter(col("vine") == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PVZCB38GOFZD|          4|            0|          1|   Y|                N|
|R1CGDATMZKD6V0|          5|            0|          0|   Y|                N|
|R18MIOEPNJ2PFF|          4|            0|          0|   Y|                N|
|R3PZ9D15P3MLT3|          4|            0|          0|   Y|                N|
|R32MW8A680GIA6|          5|            0|          0|   Y|                N|
|R1MEVMR4L53E1N|          4|            0|          1|   Y|                N|
|R10NCRGJVMP7L7|          4|            1|          1|   Y|                N|
| REJI8K2HTXLQ0|          5|            0|          1|   Y|                N|
|R3OGQ9UZQ6E1ZM|          4|            0|          0|   Y|                N|
|R3B5S9XR30OID8|          5|            0|          0|   Y|     

In [14]:
vine_unpaid_df = helpful_df.filter(col("vine") == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R2V9VX4YQ8TQ24|          5|           33|         36|   N|                Y|
|R2UCJ0GANHZVA8|          5|           21|         21|   N|                Y|
| R8MJZ3925MBK0|          5|           96|         98|   N|     

In [15]:
vine_paid_df.describe().show()

+-------+--------------+------------------+------------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+----+-----------------+
|  count|          5925|              5925|              5925|              5925|5925|             5925|
|   mean|          null| 4.187172995780591|1.6909704641350212| 2.277130801687764|null|             null|
| stddev|          null|0.9518652525936055| 11.32555778325861|12.378872621421749|null|             null|
|    min|R1008WDGJF7JGB|                 1|                 0|                 0|   Y|                N|
|    max| RZZRYZB0MPPDD|                 5|               498|               521|   Y|                Y|
+-------+--------------+------------------+------------------+------------------+----+-----------------+



In [16]:
vine_unpaid_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+-----+-----------------+
|  count|         23015|             23015|            23015|             23015|23015|            23015|
|   mean|          null|3.7365631110145556|49.16993265261786| 53.86230719096242| null|             null|
| stddev|          null|1.5848277733213492|202.8481937137797|207.13281295077013| null|             null|
|    min|R1009N0DU38ORU|                 1|               11|                21|    N|                N|
|    max| RZZQ14Q89LVMF|                 5|            26132|             26382|    N|                Y|
+-------+--------------+------------------+-----------------+------------------+-----+-----------------+

